# 1. 학습 데이터 생성

In [9]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("🤖 머신러닝 기반 응급실 혼잡도 예측 시스템")
print("=" * 60)

# ========================================
# 1. 시간대별 기본 패턴 함수
# ========================================
def get_base_congestion(hour):
    """시간대별 기본 대기시간 (분)"""
    patterns = {
        0: 15, 1: 12, 2: 10, 3: 10, 4: 12, 5: 15,
        6: 25, 7: 30, 8: 35, 9: 40, 10: 45, 11: 50,
        12: 55, 13: 50,
        14: 40, 15: 35, 16: 40, 17: 45,
        18: 60, 19: 65, 20: 60, 21: 50,
        22: 35, 23: 25
    }
    return patterns.get(hour, 30)

# ========================================
# 2. 학습용 시계열 데이터 생성
# ========================================
def generate_training_data(days=180):
    """
    6개월치 학습 데이터 생성
    다양한 변수를 고려한 합성 데이터
    """
    print(f"\n📊 {days}일치 학습 데이터 생성 중...")
    
    data = []
    start_date = datetime(2024, 4, 1)
    
    for day in range(days):
        current_date = start_date + timedelta(days=day)
        day_of_week = current_date.weekday()
        is_weekend = day_of_week >= 5
        is_holiday = is_weekend
        
        # 날씨 생성
        weather = np.random.choice(
            ['맑음', '흐림', '비', '눈'], 
            p=[0.50, 0.30, 0.15, 0.05]
        )
        
        # 기온 (계절 반영)
        month = current_date.month
        base_temp_map = {
            4: 15, 5: 20, 6: 25, 7: 28, 8: 30, 9: 25,
            10: 20, 11: 12, 12: 5, 1: 3, 2: 5, 3: 10
        }
        base_temp = base_temp_map.get(month, 20)
        temperature = base_temp + np.random.randint(-5, 6)
        
        # 24시간 데이터
        for hour in range(24):
            base_wait = get_base_congestion(hour)
            
            # 요일 영향
            if day_of_week == 0:
                base_wait *= 1.15
            elif day_of_week >= 5:
                base_wait *= 1.20
            
            # 공휴일 영향
            if is_holiday and day_of_week < 5:
                base_wait *= 1.25
            
            # 날씨 영향
            weather_multiplier = {
                '맑음': 1.0, '흐림': 1.05, '비': 1.15, '눈': 1.25
            }
            base_wait *= weather_multiplier[weather]
            
            # 기온 영향
            if temperature > 32:
                base_wait *= 1.20
            elif temperature > 28:
                base_wait *= 1.10
            elif temperature < 0:
                base_wait *= 1.25
            elif temperature < 5:
                base_wait *= 1.15
            
            # 계절 특수 상황
            if month in [12, 1, 2]:
                base_wait *= 1.10
            elif month in [7, 8]:
                base_wait *= 0.95
            
            # 랜덤 노이즈
            noise = np.random.randint(-10, 11)
            final_wait_time = int(base_wait + noise)
            final_wait_time = max(5, min(120, final_wait_time))
            
            data.append({
                'date': current_date.date(),
                'hour': hour,
                'day_of_week': day_of_week,
                'is_weekend': is_weekend,
                'is_holiday': is_holiday,
                'month': month,
                'weather': weather,
                'temperature': temperature,
                'wait_time': final_wait_time
            })
    
    df = pd.DataFrame(data)
    print(f"✅ 학습 데이터 생성 완료: {len(df):,}개 샘플")
    return df

# 데이터 생성 실행
df_train = generate_training_data(days=180)

# ========================================
# 3. 데이터 확인
# ========================================
print("\n📋 데이터 샘플:")
print(df_train.head(10))

print("\n📊 대기시간 통계:")
print(df_train['wait_time'].describe())

print("\n🌤️ 날씨별 평균 대기시간:")
print(df_train.groupby('weather')['wait_time'].mean().round(1))

print("\n📅 요일별 평균 대기시간:")
days_kr = ['월', '화', '수', '목', '금', '토', '일']
weather_by_day = df_train.groupby('day_of_week')['wait_time'].mean().round(1)
for idx, wait in weather_by_day.items():
    print(f"  {days_kr[idx]}요일: {wait:.1f}분")

print("\n⏰ 시간대별 평균 대기시간 (상위 5개):")
hourly = df_train.groupby('hour')['wait_time'].mean().sort_values(ascending=False).head(5)
for hour, wait in hourly.items():
    print(f"  {hour:02d}시: {wait:.1f}분")

print("\n✅ 셀 1 완료! 다음 셀을 실행하세요.")

🤖 머신러닝 기반 응급실 혼잡도 예측 시스템

📊 180일치 학습 데이터 생성 중...
✅ 학습 데이터 생성 완료: 4,320개 샘플

📋 데이터 샘플:
         date  hour  day_of_week  is_weekend  is_holiday  month weather  \
0  2024-04-01     0            0       False       False      4       비   
1  2024-04-01     1            0       False       False      4       비   
2  2024-04-01     2            0       False       False      4       비   
3  2024-04-01     3            0       False       False      4       비   
4  2024-04-01     4            0       False       False      4       비   
5  2024-04-01     5            0       False       False      4       비   
6  2024-04-01     6            0       False       False      4       비   
7  2024-04-01     7            0       False       False      4       비   
8  2024-04-01     8            0       False       False      4       비   
9  2024-04-01     9            0       False       False      4       비   

   temperature  wait_time  
0           20         11  
1           20         14  
2   

# 2. 머신러닝 모델 학습

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle
import os

print("\n" + "=" * 60)
print("🤖 머신러닝 모델 학습")
print("=" * 60)

# ========================================
# 1. 특징 엔지니어링
# ========================================
print("\n🔧 특징 엔지니어링...")

# 날씨를 숫자로 변환 (원-핫 인코딩)
df_encoded = pd.get_dummies(df_train, columns=['weather'], prefix='weather')

# 특징과 타겟 분리
feature_columns = [
    'hour', 'day_of_week', 'is_weekend', 'is_holiday', 
    'month', 'temperature',
    'weather_맑음', 'weather_눈', 'weather_비', 'weather_흐림'
]

X = df_encoded[feature_columns]
y = df_encoded['wait_time']

print(f"✅ 특징 변수: {len(feature_columns)}개")
print(f"✅ 샘플 수: {len(X):,}개")

# ========================================
# 2. 학습/테스트 데이터 분할
# ========================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\n📚 학습 데이터: {len(X_train):,}개")
print(f"📖 테스트 데이터: {len(X_test):,}개")

# ========================================
# 3. Random Forest 모델 학습
# ========================================
print("\n🌲 Random Forest 모델 학습 중...")

model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)
print("✅ 모델 학습 완료!")

# ========================================
# 4. 모델 평가
# ========================================
print("\n📊 모델 평가:")

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)

rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

accuracy_train = (1 - mae_train/y_train.mean()) * 100
accuracy_test = (1 - mae_test/y_test.mean()) * 100

print(f"\n[학습 데이터 성능]")
print(f"  MAE (평균 절대 오차): {mae_train:.2f}분")
print(f"  RMSE (평균 제곱근 오차): {rmse_train:.2f}분")
print(f"  R² (결정계수): {r2_train:.3f}")
print(f"  정확도: {accuracy_train:.1f}%")

print(f"\n[테스트 데이터 성능]")
print(f"  MAE (평균 절대 오차): {mae_test:.2f}분")
print(f"  RMSE (평균 제곱근 오차): {rmse_test:.2f}분")
print(f"  R² (결정계수): {r2_test:.3f}")
print(f"  정확도: {accuracy_test:.1f}%")

if accuracy_test >= 85:
    print(f"\n🎉 목표 달성! 예측 정확도 {accuracy_test:.1f}% (목표: 85% 이상)")
else:
    print(f"\n⚠️ 목표 미달성: {accuracy_test:.1f}% (목표: 85%)")

# ========================================
# 5. 특징 중요도 분석
# ========================================
print("\n🎯 특징 중요도 (Top 5):")
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

for idx, row in feature_importance.head(5).iterrows():
    print(f"  {row['feature']:20s}: {row['importance']:.3f}")

# ========================================
# 6. 모델 저장
# ========================================
models_dir = 'D:/emergency_room/data-analysis/models'
os.makedirs(models_dir, exist_ok=True)

model_path = f'{models_dir}/congestion_model.pkl'
features_path = f'{models_dir}/feature_columns.pkl'

with open(model_path, 'wb') as f:
    pickle.dump(model, f)

with open(features_path, 'wb') as f:
    pickle.dump(feature_columns, f)

print(f"\n💾 모델 저장 완료:")
print(f"  - {model_path}")
print(f"  - {features_path}")

print("\n✅ 모델 학습 완료! 다음 셀을 실행하세요.")


🤖 머신러닝 모델 학습

🔧 특징 엔지니어링...
✅ 특징 변수: 10개
✅ 샘플 수: 4,320개

📚 학습 데이터: 3,456개
📖 테스트 데이터: 864개

🌲 Random Forest 모델 학습 중...
✅ 모델 학습 완료!

📊 모델 평가:

[학습 데이터 성능]
  MAE (평균 절대 오차): 3.52분
  RMSE (평균 제곱근 오차): 4.24분
  R² (결정계수): 0.959
  정확도: 91.3%

[테스트 데이터 성능]
  MAE (평균 절대 오차): 5.66분
  RMSE (평균 제곱근 오차): 6.70분
  R² (결정계수): 0.904
  정확도: 86.3%

🎉 목표 달성! 예측 정확도 86.3% (목표: 85% 이상)

🎯 특징 중요도 (Top 5):
  hour                : 0.881
  temperature         : 0.032
  day_of_week         : 0.028
  weather_눈           : 0.015
  month               : 0.012

💾 모델 저장 완료:
  - D:/emergency_room/data-analysis/models/congestion_model.pkl
  - D:/emergency_room/data-analysis/models/feature_columns.pkl

✅ 모델 학습 완료! 다음 셀을 실행하세요.


# 3. 46개 병원 예측 데이터 생성

In [7]:
print("\n" + "=" * 60)
print("🏥 46개 병원 × 24시간 예측 데이터 생성")
print("=" * 60)

# ========================================
# 1. 병원 데이터 로드
# ========================================
hospitals_path = 'D:/emergency_room/data-analysis/data/raw/busan_hospitals_static.csv'
hospitals = pd.read_csv(hospitals_path)

print(f"\n✅ 병원 데이터 로드: {len(hospitals)}개 병원")

# ========================================
# 2. 병원 특성별 조정 함수
# ========================================
def adjust_by_hospital(base_time, emergency_level, beds_total):
    """병원 특성을 반영한 대기시간 조정"""
    level_multiplier = {
        '권역응급의료센터': 1.3,
        '지역응급의료센터': 1.1,
        '지역응급의료기관': 0.9,
        '응급의료기관': 0.8
    }
    
    multiplier = level_multiplier.get(emergency_level, 1.0)
    adjusted = base_time * multiplier
    
    if pd.notna(beds_total):
        if beds_total > 500:
            adjusted *= 0.85
        elif beds_total > 300:
            adjusted *= 0.95
        elif beds_total < 200:
            adjusted *= 1.1
    
    return int(adjusted)

def get_congestion_info(wait_time):
    """대기시간 → 혼잡도 레벨 & 색상"""
    if wait_time < 30:
        return '여유', 'green'
    elif wait_time < 60:
        return '보통', 'yellow'
    else:
        return '혼잡', 'red'

# ========================================
# 3. 예측 데이터 생성
# ========================================
print("\n🔮 예측 데이터 생성 중...")

# 현재 날짜 정보
now = datetime.now()
current_day = now.weekday()
is_weekend = current_day >= 5
current_month = now.month

# 현재 날씨 (가정)
current_weather = '맑음'
current_temp = 20

predictions = []

for _, hospital in hospitals.iterrows():
    for hour in range(24):
        # 예측용 특징 데이터 생성
        X_pred_dict = {
            'hour': hour,
            'day_of_week': current_day,
            'is_weekend': is_weekend,
            'is_holiday': False,
            'month': current_month,
            'temperature': current_temp,
            'weather_맑음': 1 if current_weather == '맑음' else 0,
            'weather_눈': 1 if current_weather == '눈' else 0,
            'weather_비': 1 if current_weather == '비' else 0,
            'weather_흐림': 1 if current_weather == '흐림' else 0,
        }
        
        X_pred = pd.DataFrame([X_pred_dict])[feature_columns]
        
        # 모델 예측
        predicted_time = int(model.predict(X_pred)[0])
        
        # 병원 특성 반영
        adjusted_time = adjust_by_hospital(
            predicted_time,
            hospital['emergency_level'],
            hospital['beds_total']
        )
        
        # 혼잡도 레벨 결정
        level, color = get_congestion_info(adjusted_time)
        
        predictions.append({
            'hospital_id': hospital['id'],
            'prediction_hour': hour,
            'predicted_wait_time': adjusted_time,
            'congestion_level': level,
            'congestion_color': color
        })

df_predictions = pd.DataFrame(predictions)

print(f"✅ 예측 데이터 생성 완료: {len(df_predictions):,}개")

# ========================================
# 4. 통계 확인
# ========================================
print("\n📊 예측 결과 통계:")
print(df_predictions['congestion_level'].value_counts())

print("\n⏰ 시간대별 평균 예측 대기시간:")
hourly_pred = df_predictions.groupby('prediction_hour')['predicted_wait_time'].mean()
for hour in [6, 9, 12, 15, 18, 21]:
    print(f"  {hour:02d}시: {hourly_pred[hour]:.1f}분")

# ========================================
# 5. CSV 저장
# ========================================
output_path = 'D:/emergency_room/data-analysis/data/simulated/ml_emergency_predictions.csv'
df_predictions.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n💾 CSV 저장 완료: {output_path}")
print("\n✅ 예측 데이터 생성 완료! 다음 셀에서 MySQL에 삽입합니다.")


🏥 46개 병원 × 24시간 예측 데이터 생성

✅ 병원 데이터 로드: 46개 병원

🔮 예측 데이터 생성 중...
✅ 예측 데이터 생성 완료: 1,104개

📊 예측 결과 통계:
congestion_level
보통    642
여유    383
혼잡     79
Name: count, dtype: int64

⏰ 시간대별 평균 예측 대기시간:
  06시: 24.0분
  09시: 38.0분
  12시: 51.4분
  15시: 38.0분
  18시: 60.9분
  21시: 46.0분

💾 CSV 저장 완료: D:/emergency_room/data-analysis/data/simulated/ml_emergency_predictions.csv

✅ 예측 데이터 생성 완료! 다음 셀에서 MySQL에 삽입합니다.


# 4. MySQL에 삽입

In [11]:
import mysql.connector
from mysql.connector import Error

print("\n" + "=" * 60)
print("💾 MySQL에 예측 데이터 삽입")
print("=" * 60)

# ========================================
# 1. CSV 로드
# ========================================
csv_path = 'D:/emergency_room/data-analysis/data/simulated/ml_emergency_predictions.csv'
df = pd.read_csv(csv_path)

print(f"\n✅ CSV 로드 완료: {len(df)}개 데이터")

# ========================================
# 2. MySQL 연결
# ========================================
try:
    connection = mysql.connector.connect(
        host='localhost',
        port=3306,
        database='emergency_room',
        user='root',
        password='1234'  # ← 🔑 비밀번호!
    )
    
    if connection.is_connected():
        print(f"✅ MySQL 연결 성공!")
        
        cursor = connection.cursor()
        
        # ========================================
        # 3. 기존 데이터 삭제
        # ========================================
        cursor.execute("DELETE FROM emergency_predictions")
        connection.commit()
        print(f"✅ 기존 데이터 삭제 완료")
        
        # ========================================
        # 4. 새 데이터 삽입
        # ========================================
        insert_query = """
        INSERT INTO emergency_predictions 
        (hospital_id, prediction_hour, predicted_wait_time, congestion_level, congestion_color)
        VALUES (%s, %s, %s, %s, %s)
        """
        
        data_list = df.values.tolist()
        
        print(f"\n📥 데이터 삽입 중...")
        cursor.executemany(insert_query, data_list)
        connection.commit()
        
        print(f"✅ {cursor.rowcount}개 데이터 삽입 완료!")
        
        # ========================================
        # 5. 확인
        # ========================================
        cursor.execute("SELECT COUNT(*) FROM emergency_predictions")
        total = cursor.fetchone()[0]
        print(f"\n📊 테이블 총 레코드 수: {total}")
        
        cursor.execute("""
            SELECT congestion_level, COUNT(*) 
            FROM emergency_predictions 
            GROUP BY congestion_level
        """)
        
        print(f"\n혼잡도 분포:")
        for row in cursor.fetchall():
            print(f"  {row[0]}: {row[1]}개")
        
        cursor.close()
        print(f"\n✅ MySQL 삽입 완료!")

except Error as e:
    print(f"❌ MySQL 오류: {e}")

finally:
    if connection.is_connected():
        connection.close()
        print(f"\nMySQL 연결 종료")


💾 MySQL에 예측 데이터 삽입

✅ CSV 로드 완료: 1104개 데이터
✅ MySQL 연결 성공!
✅ 기존 데이터 삭제 완료

📥 데이터 삽입 중...
✅ 1104개 데이터 삽입 완료!

📊 테이블 총 레코드 수: 1104

혼잡도 분포:
  여유: 383개
  보통: 642개
  혼잡: 79개

✅ MySQL 삽입 완료!

MySQL 연결 종료
